
# Alesmodel: Plotting sections and maps.

![foo](ales.png)

## Setting up enviroment and pulling data

In [ ]:
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')
import matplotlib
matplotlib.rcParams['figure.figsize'] = (20.0, 15.0)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pooch

os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=cuda"
# Importing GemPy
import gempy as gp

In [ ]:
# Pull raw data
model_input_files = pooch.retrieve(
    url="https://github.com/cgre-aachen/gempy_data/raw/master/"
        "data/input_data/AlesModel/AlesModel.zip",
    known_hash="38290d8343a60784265f8d873731f22a22cf0d340232e50dda323b265e95dc1e",
    processor=pooch.Unzip()  
)
 
model_input_files

In [ ]:
pd.read_csv(model_input_files[0])

## Initializing model

In [ ]:
resolution = [100, 100, 100]
extent = np.array([729550.0, 751500.0, 1913500.0, 1923650.0, -1800.0, 800.0])
geo_model = gp.create_model('Alesmodel')
gp.init_data(geo_model, extent=extent, resolution=resolution,
             path_i=model_input_files[0],
             path_o=model_input_files[1])

In [ ]:
gp.set_interpolator(geo_model,
                    output=['geology'], compile_theano=True,
                    theano_optimizer='fast_run', dtype='float64',
                    verbose=[])

## Setting up geological relationships



In [ ]:
gp.map_stack_to_surfaces(geo_model, {'fault_left': ('fault_left'),
                                     'fault_right': ('fault_right'),
                                     'fault_lr': ('fault_lr'),
                                     'Trias_Series': ('TRIAS', 'LIAS'),
                                     'Carbon_Series': ('CARBO'),
                                     'Basement_Series': ('basement')}, remove_unused_series=True)

In [ ]:
colordict = {'LIAS': '#015482', 'TRIAS': '#9f0052', 'CARBO': '#ffbe00', 'basement': '#728f02',
             'fault_left': '#2a2a2a', 'fault_right': '#545454', 'fault_lr': '#a5a391'}
geo_model.surfaces.colors.change_colors(colordict)

In [ ]:
a = gp.plot_2d(geo_model, direction='y')

## Faults




In [ ]:
geo_model.set_is_fault(['fault_right', 'fault_left', 'fault_lr'], change_color=True)

### Topography




In [ ]:
geo_model.set_topography(source='gdal', filepath=model_input_files[2])

In [ ]:
gp.plot_3d(geo_model, show_topography=True)

## Compute model

In [ ]:
geo_model.update_to_interpolator()
_ = gp.compute_model(geo_model, compute_mesh=True, compute_mesh_options={'rescale': False})

In [ ]:
gp.plot_2d(geo_model, cell_number=[4], direction=['y'], show_topography=True,
           show_data=True)

In [ ]:
gp.plot_2d(geo_model, section_names=['topography'], show_data=False,
           show_boundaries=False)

In [ ]:
gp.plot_3d(geo_model, plotter_type="background")